#### Installing Libraries

In [1]:
!pip install ffmpeg | tee /dev/null > /dev/null
!pip install num2words | tee /dev/null > /dev/null
!pip install pydub | tee /dev/null > /dev/null
!pip install yt-dlp | tee /dev/null > /dev/null
!pip install pdfminer.six | tee /dev/null > /dev/null

#### Imports

In [2]:
from google.colab import drive
import requests
import json
import pprint
import subprocess
import shutil
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path
from num2words import num2words
from pydub import AudioSegment
from concurrent.futures import ProcessPoolExecutor
import time
from pydub.utils import mediainfo
import os
from pdfminer.high_level import extract_text
import string

In [4]:
n_files = 1 ## number of files you want to download -1 for all

### Imports

#Task-1


### Mounting Drive

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


### Requesting for Video,Transcript,Audio Urls

In [7]:
course_id = input("Enter the URL: ")
url = f"https://tools.nptel.ac.in/npteldata/downloads.php?id={course_id}"


response = requests.get(url)

if response.status_code == 200:
    try:
        data = response.json()
        # print(data)
    except json.JSONDecodeError:
        print("Received a successful response, but the content is not valid JSON.")
        print("Response content:", response.text)  # Print the raw response for debugging
elif response.status_code == 404:
    print("Error: The requested resource was not found (404).")
elif response.status_code == 500:
    print("Error: Internal Server Error (500). The server encountered an unexpected condition.")
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")
    print(f"Reason: {response.reason}") # Print the reason for the error.

Enter the URL: 101105084


In [8]:
print('course url: ' ,  f"https://tools.nptel.ac.in/npteldata/downloads.php?id={course_id}")

course url:  https://tools.nptel.ac.in/npteldata/downloads.php?id=101105084


In [9]:
data.get('data').keys()

dict_keys(['title', 'course_id', 'is_public', 'course_downloads', 'transcripts', 'audio', 'books', 'assignments'])

In [10]:
course_title = data['data']['title'] ## title of the course
lectures_data = data['data']['course_downloads']  ## video url of the course
lectures_transcript = data['data']['transcripts'] ## lecture transcript of the course
print('Course Title' , course_title)

Course Title NOC:Aircraft Structures - I


In [11]:
lectures_data[:5] # one instance of lectures_data

[{'title': 'Lecture 01: Introduction',
  'url': 'https://media.dev.nptel.ac.in/content/mp4/101/105/101105084/MP4/mod01lec01.mp4',
  'mimetype': 'video/mp4',
  'lesson_id': '1'},
 {'title': "Lecture 02: Introduction (Cont'd)",
  'url': 'https://media.dev.nptel.ac.in/content/mp4/101/105/101105084/MP4/mod01lec02.mp4',
  'mimetype': 'video/mp4',
  'lesson_id': '2'},
 {'title': "Lecture 03: Introduction (Cont'd)",
  'url': 'https://media.dev.nptel.ac.in/content/mp4/101/105/101105084/MP4/mod01lec03.mp4',
  'mimetype': 'video/mp4',
  'lesson_id': '3'},
 {'title': 'Lecture 04: Load Encountered by a Typical Aircraft',
  'url': 'https://media.dev.nptel.ac.in/content/mp4/101/105/101105084/MP4/mod01lec04.mp4',
  'mimetype': 'video/mp4',
  'lesson_id': '4'},
 {'title': 'Lecture 05: Conceptual Structural Details of a Typical Aircraft',
  'url': 'https://media.dev.nptel.ac.in/content/mp4/101/105/101105084/MP4/mod01lec05.mp4',
  'mimetype': 'video/mp4',
  'lesson_id': '5'}]

In [12]:
pprint.pprint(lectures_transcript[0])

{'downloads': [{'language': 'english-Verified',
                'url': 'https://drive.google.com/file/d/1y8qq758GzC0A5iFLte2E1CoWumJ09foR/view'},
               {'language': 'bengali', 'url': None},
               {'language': 'gujarati', 'url': None},
               {'language': 'hindi', 'url': None},
               {'language': 'kannada', 'url': None},
               {'language': 'malayalam', 'url': None},
               {'language': 'marathi', 'url': None},
               {'language': 'tamil', 'url': None},
               {'language': 'telugu', 'url': None},
               {'language': 'punjabi', 'url': None},
               {'language': 'odiya', 'url': None},
               {'language': 'assamese', 'url': None}],
 'lesson_id': '1',
 'title': 'Lecture 01: Introduction'}


### Creating Directory for Lectures and Downloading Lecture.

In [14]:
video_path = data['data']['title'] + '_videos' 
os.makedirs(video_path, exist_ok=True)  
for lecture in lectures_data[:n_files]:
    url = lecture['url']
    filename = lecture['title']

    output_path = os.path.join(video_path, f"{filename}.mp4")  

    try:
        subprocess.run(
            ["yt-dlp", "-o", output_path, url], check=True
        )
        print(f"Downloaded: {output_path}")
    except subprocess.CalledProcessError as e:
        print(f"Error downloading {url}: {e}")

Downloaded: NOC:Aircraft Structures - I_videos/Lecture 01: Introduction.mp4


### Making Directories for Transcript Pdf For each language

In [15]:
def create_folder(path):
    try:
        if not os.path.exists(path):
            os.mkdir(path)
        else :
            shutil.rmtree(path)
            os.mkdir(path)

        print(f"Created folder: {path}")
    except FileExistsError:
        print(f"Folder already exists: {path}")

In [16]:
transcript_pdf_path = course_title + '_transcripts_pdfs'
create_folder(transcript_pdf_path)


languages_transcript = {}
for lang in lectures_transcript[0]['downloads']:
    languages_transcript[lang['language']] = []


languages_transcript
path = os.path.join(course_title + '_transcripts_pdfs')


create_folder(path)


for language in languages_transcript.keys() :
    folder_path = os.path.join(path, f"{language}_transcripts_pdfs")
    try :
        create_folder(folder_path)
    except Exception as e:
        print(f"Error creating folder {folder_path}: {e}")

Created folder: NOC:Aircraft Structures - I_transcripts_pdfs
Created folder: NOC:Aircraft Structures - I_transcripts_pdfs
Created folder: NOC:Aircraft Structures - I_transcripts_pdfs/english-Verified_transcripts_pdfs
Created folder: NOC:Aircraft Structures - I_transcripts_pdfs/bengali_transcripts_pdfs
Created folder: NOC:Aircraft Structures - I_transcripts_pdfs/gujarati_transcripts_pdfs
Created folder: NOC:Aircraft Structures - I_transcripts_pdfs/hindi_transcripts_pdfs
Created folder: NOC:Aircraft Structures - I_transcripts_pdfs/kannada_transcripts_pdfs
Created folder: NOC:Aircraft Structures - I_transcripts_pdfs/malayalam_transcripts_pdfs
Created folder: NOC:Aircraft Structures - I_transcripts_pdfs/marathi_transcripts_pdfs
Created folder: NOC:Aircraft Structures - I_transcripts_pdfs/tamil_transcripts_pdfs
Created folder: NOC:Aircraft Structures - I_transcripts_pdfs/telugu_transcripts_pdfs
Created folder: NOC:Aircraft Structures - I_transcripts_pdfs/punjabi_transcripts_pdfs
Created fol

In [ ]:
# !rm -rf '/content/NOC:Deep Learning_transcripts_pdfs'

### Downloading pdf Transcript

In [17]:
import os
import requests
from concurrent.futures import ThreadPoolExecutor

transcript_pdf_path = course_title + '_transcripts_pdfs'

# Function to download a single file
def download_file(download_data):
    url, output_path = download_data
    try:
        # Make sure the folder exists
        os.makedirs(os.path.dirname(output_path), exist_ok=True)

        # Download the file
        response = requests.get(url)
        response.raise_for_status()  # Ensure we got a valid response

        with open(output_path, 'wb') as f:
            f.write(response.content)

        print(f"Downloaded: {output_path}")
    except Exception as e:
        print(f"Error downloading {url}: {e}")

download_list = []
for lecture in lectures_transcript[:n_files]:
    lecture_title = lecture['title']
    for lang in lecture['downloads']:
        url_view = lang['url']
        language = lang['language']

        if url_view:
            url_raw_comp = url_view.split('/')
            file_id = url_raw_comp[-2]
            url = f"https://drive.google.com/uc?id={file_id}&export=download"

            output_path = os.path.join(transcript_pdf_path, f"{language}_transcripts_pdfs", f"{lecture_title}.pdf")
            download_list.append((url, output_path))

# Use ThreadPoolExecutor for parallel downloads
with ThreadPoolExecutor(max_workers=10) as executor:
    executor.map(download_file, download_list)

Downloaded: NOC:Aircraft Structures - I_transcripts_pdfs/english-Verified_transcripts_pdfs/Lecture 01: Introduction.pdf


In [10]:
# /content/NOC:Deep Learning_transcripts_pdfs
path

'NOC:Deep Learning_transcripts_pdfs'

### Checking number of pdf file for each language

In [18]:
# root_folder = '/content/NOC:Deep Learning_videos_transcripts_pdfs'
transcript_pdf_path = course_title + '_transcripts_pdfs'


for folder_name in os.listdir(transcript_pdf_path):
    folder_path = os.path.join(transcript_pdf_path, folder_name)

    if os.path.isdir(folder_path):
        # Count the number of files in the folder
        file_count = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])
        print(f"Folder: {folder_name}, Number of files: {file_count}")

Folder: kannada_transcripts_pdfs, Number of files: 0
Folder: malayalam_transcripts_pdfs, Number of files: 0
Folder: hindi_transcripts_pdfs, Number of files: 0
Folder: odiya_transcripts_pdfs, Number of files: 0
Folder: marathi_transcripts_pdfs, Number of files: 0
Folder: punjabi_transcripts_pdfs, Number of files: 0
Folder: assamese_transcripts_pdfs, Number of files: 0
Folder: gujarati_transcripts_pdfs, Number of files: 0
Folder: english-Verified_transcripts_pdfs, Number of files: 1
Folder: telugu_transcripts_pdfs, Number of files: 0
Folder: bengali_transcripts_pdfs, Number of files: 0
Folder: tamil_transcripts_pdfs, Number of files: 0


### Download Audio file

asynchronous requests

In [19]:
# import os
# import asyncio
# import aiohttp
# import subprocess

# audios_path = course_title + '____test_audios_mp3'  # Define the path for the audio files
# create_folder(audios_path)  # Create the folder if it doesn't exist

# # Function to run yt-dlp for audio extraction
# def download_audio_with_yt_dlp(url, output_path):
#     try:
#         subprocess.run(
#             [
#                 "yt-dlp",
#                 "-x",  # Extract audio only
#                 "--audio-format", "mp3",  # Specify audio format (e.g., mp3)
#                 "-o", output_path,  # Specify output path
#                 url
#             ],
#             check=True
#         )
#         print(f"Audio downloaded: {output_path}")
#     except subprocess.CalledProcessError as e:
#         print(f"Error downloading {url}: {e}")

# # Asynchronous function to handle each lecture download
# async def download_audio_async(session, url, output_path):
#     try:
#         # Perform the download using yt-dlp (synchronously)
#         download_audio_with_yt_dlp(url, output_path)
#     except Exception as e:
#         print(f"Error downloading {url}: {e}")

# # Asynchronous function to download all audios in parallel
# async def download_all_audios():
#     async with aiohttp.ClientSession() as session:
#         tasks = []
#         for lecture in lectures_data:
#             url = lecture['url']
#             filename = lecture['title']
#             output_path = os.path.join(audios_path, f"{filename}.mp3")
#             task = download_audio_async(session, url, output_path)
#             tasks.append(task)
#         await asyncio.gather(*tasks)

# # Directly run the asynchronous download function in the existing event loop (for Colab)
# await download_all_audios()

In [20]:
# !rm -rf '/content/NOC:Deep Learning_test_audios_mp3'
# !rm -rf '/content/NOC:Deep Learning_test_audios_mp3'
# !rm -rf '/content/NOC:Deep Learning____test_audios_mp3'

In [21]:
import os
import subprocess

# Define the path for the audio files
audios_path = course_title + '_test_audios_mp3'

# Create the folder if it doesn't exist
create_folder(audios_path)

# Function to run yt-dlp for audio extraction
def download_audio_with_yt_dlp(url, output_path):
    try:
        subprocess.run(
            [
                "yt-dlp",
                "-x",  
                "--audio-format", "mp3", 
                "-o", output_path, 
                url
            ],
            check=True
        )
        print(f"Audio downloaded: {output_path}")
    except subprocess.CalledProcessError as e:
        print(f"Error downloading {url}: {e}")



def download_all_audios_sequentially():
    for lecture in lectures_data[:n_files]:
        url = lecture['url']
        filename = lecture['title']
        output_path = os.path.join(audios_path, f"{filename}.mp3")
        download_audio_with_yt_dlp(url, output_path)


download_all_audios_sequentially()

Created folder: NOC:Aircraft Structures - I_test_audios_mp3
Audio downloaded: NOC:Aircraft Structures - I_test_audios_mp3/Lecture 01: Introduction.mp3


## TASK - 2
### Converting the mp3 to  WAV format, with 16KHz sampling rate, mono channel format.

In [22]:
!cat "/content/drive/MyDrive/preprocess_audios.sh"

#!/bin/bash

# Ensure the script exits on errors and undefined variables
set -e
set -u

# Function to display usage instructions
usage() {
    echo "Usage: $0 <input_directory> <output_directory> <num_cpus>"
    exit 1
}

# Check for correct number of arguments
if [ "$#" -ne 3 ]; then
    usage
fi

# Read user inputs
INPUT_DIR="$1"
OUTPUT_DIR="$2"
NUM_CPUS="$3"

# Check if input directory exists
if [ ! -d "$INPUT_DIR" ]; then
    echo "Error: Input directory '$INPUT_DIR' does not exist."
    exit 1
fi

# Create output directory if it doesn't exist
mkdir -p "$OUTPUT_DIR"

# Function to process a single file
process_file() {
    local input_file="$1"
    local output_dir="$2"
    local base_name
    base_name=$(basename "$input_file" | sed 's/\.[^.]*$//') # Strip extension
    local output_file="${output_dir}/${base_name}.wav"

    # Convert audio file to WAV format with 16KHz sampling rate and mono channel
    ffmpeg -i "$input_file" -ar 16000 -ac 1 "$output_file" -y
}

# Export the fun

In [23]:
output_dir = course_title + '_test_audios_wav'
create_folder(output_dir)

Created folder: NOC:Aircraft Structures - I_test_audios_wav


In [25]:
!chmod +x /content/drive/MyDrive/preprocess_audios.sh
!/content/drive/MyDrive/preprocess_audios.sh '/content/NOC:Aircraft Structures - I_test_audios_mp3' '/content/NOC:Aircraft Structures - I_test_audios_wav' 2

xargs: warning: options --max-args and --replace/-I/-i are mutually exclusive, ignoring previous --max-args value
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --

### Preprocessing sound

#### I am clipping the sound of Instruments before 10 sec and last 30 sec.

In [26]:
start_time = time.time()
input_dir = '/content/' + course_title + '_test_audios_wav'
output_dir = "/content/Audio_trimmed_wav"


create_folder(output_dir)


def process_audio(filename):
    if filename.endswith(".wav"):
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, f"{filename}")

        # Load the audio file
        audio = AudioSegment.from_wav(input_path)

        duration_ms = len(audio)  # Total duration in milliseconds
        start_trim = 10 * 1000  # 10 seconds in milliseconds
        end_trim = duration_ms - (30 * 1000)  # Remove last 30 seconds

        # Trimming
        trimmed_audio = audio[start_trim:end_trim]

        trimmed_audio.export(output_path, format="wav")
        print(f"{filename} -> {output_path}")



wav_files = [filename for filename in os.listdir(input_dir) if filename.endswith(".wav")]


with ProcessPoolExecutor() as executor:
    executor.map(process_audio, wav_files)

print("Trimming complete!")
print("--- %s seconds ---" % (time.time() - start_time))

Created folder: /content/Audio_trimmed_wav
Lecture 01: Introduction.wav -> /content/Audio_trimmed_wav/Lecture 01: Introduction.wav
Trimming complete!
--- 0.1356062889099121 seconds ---


## TASK - 3

### Extracting text From pdfs files and preprocessing it.



*   Extracting text
*   Removing punctation
*   lower case




In [27]:



input_dir = "/content/" + course_title + "_transcripts_pdfs"
output_base_dir = "/content/processed_txt_files"

create_folder(output_base_dir)

# Function to process text: lowercase, remove punctuation, and convert digits to words
def process_text(text):
    text = text.lower()

    text = text.translate(str.maketrans("", "", string.punctuation))

    words = text.split()
    processed_words = []

    for word in words:
        if word.isdigit():  
            word = num2words(word)  
        processed_words.append(word)

    return " ".join(processed_words)

def extract_text_from_pdf(pdf_path):
    try:
        return extract_text(pdf_path)
    except Exception as e:
        print(f"Error extracting text from {pdf_path}: {e}")
        return ""


def process_language_directory(language_dir, output_language_dir):
    os.makedirs(output_language_dir, exist_ok=True)  # Ensure output language directory exists

    for filename in os.listdir(language_dir):
        if filename.endswith(".pdf"):  # Only process PDF files
            pdf_path = os.path.join(language_dir, filename)
            text = extract_text_from_pdf(pdf_path)

            processed_text = process_text(text)

            # Save the processed text to a .txt file
            output_filename = os.path.splitext(filename)[0] + ".txt"  # Change extension to .txt
            output_path = os.path.join(output_language_dir, output_filename)

            with open(output_path, "w", encoding="utf-8") as file:
                file.write(processed_text)

            print(f"Processed: {pdf_path} -> {output_path}")

def process_all_languages(input_dir, output_base_dir):
    for language in os.listdir(input_dir): # all language pdfs
        language_dir = os.path.join(input_dir, language) #
        output_language_dir = os.path.join(output_base_dir, language + "_txt_files")

        if os.path.isdir(language_dir):  # Check if it's a directory
            print(f"Processing language: {language}")
            process_language_directory(language_dir, output_language_dir)

# Start processing
process_all_languages(input_dir, output_base_dir)

print("Processing complete!")

Created folder: /content/processed_txt_files
Processing language: kannada_transcripts_pdfs
Processing language: malayalam_transcripts_pdfs
Processing language: hindi_transcripts_pdfs
Processing language: odiya_transcripts_pdfs
Processing language: marathi_transcripts_pdfs
Processing language: punjabi_transcripts_pdfs
Processing language: assamese_transcripts_pdfs
Processing language: gujarati_transcripts_pdfs
Processing language: english-Verified_transcripts_pdfs
Processed: /content/NOC:Aircraft Structures - I_transcripts_pdfs/english-Verified_transcripts_pdfs/Lecture 01: Introduction.pdf -> /content/processed_txt_files/english-Verified_transcripts_pdfs_txt_files/Lecture 01: Introduction.txt
Processing language: telugu_transcripts_pdfs
Processing language: bengali_transcripts_pdfs
Processing language: tamil_transcripts_pdfs
Processing complete!


## TASK 4

#### Download data in the format for each language available.
```
{
"audio_filepath": "data/courses/106106184/audio/lec_1_2.wav",
"duration": 824.4,
"text": "some text file."
}

```



In [28]:



audio_dir = "/content/Audio_trimmed_wav"  # Directory containing audio files
transcript_base_dir = "/content/processed_txt_files"  # Base directory containing transcript files
output_base_dir = "/content/json_output"  # Output base directory for the JSON files

def get_audio_duration(audio_file):
    info = mediainfo(audio_file)
    return float(info["duration"])


def create_jsonl_for_language(language, transcript_dir, audio_dir, output_language_dir):
    create_folder(output_language_dir)

    jsonl_manifest_path = os.path.join(output_language_dir, f"{language}_train_manifest.jsonl")

    with open(jsonl_manifest_path, "w", encoding="utf-8") as jsonl_file:
        # Iterate through audio files
        for audio_file in os.listdir(audio_dir):
            if audio_file.endswith(".wav"):
                # Construct audio file path
                audio_path = os.path.join(audio_dir, audio_file)

                # Get the corresponding transcript file
                base_name = os.path.splitext(audio_file)[0].strip()  # Strip spaces from base name
                transcript_path = os.path.join(transcript_dir, f"{base_name}.txt")

                # Check if the transcript file exists
                if os.path.exists(transcript_path):
                    print(f"Processing: {audio_path} with {transcript_path}")

                    with open(transcript_path, "r", encoding="utf-8") as transcript_file:
                        text = transcript_file.read().strip()

                    duration = get_audio_duration(audio_path)

                    json_object = {
                        "audio_filepath": audio_path,
                        "duration": duration,
                        "text": text,
                    }

                    jsonl_file.write(json.dumps(json_object) + "\n")


# Function to process all languages
def process_all_languages(input_dir, audio_dir, output_base_dir):
    for language in os.listdir(input_dir):  # Iterate through all language directories
        language_dir = os.path.join(input_dir, language)  # Language directory path
        output_language_dir = os.path.join(output_base_dir, language)  # Output folder for the language JSON files
        print(output_language_dir , 'output dir')
        print(language_dir , 'languate dir')
        if os.path.isdir(language_dir):  # Check if it's a directory
            print(f"Processing language: {language}")
            create_jsonl_for_language(language, language_dir, audio_dir, output_language_dir)


create_folder(output_base_dir)


process_all_languages(transcript_base_dir, audio_dir, output_base_dir)
print(f"Training manifest files created successfully in '{output_base_dir}'!")

Created folder: /content/json_output
/content/json_output/punjabi_transcripts_pdfs_txt_files output dir
/content/processed_txt_files/punjabi_transcripts_pdfs_txt_files languate dir
Processing language: punjabi_transcripts_pdfs_txt_files
Created folder: /content/json_output/punjabi_transcripts_pdfs_txt_files
/content/json_output/telugu_transcripts_pdfs_txt_files output dir
/content/processed_txt_files/telugu_transcripts_pdfs_txt_files languate dir
Processing language: telugu_transcripts_pdfs_txt_files
Created folder: /content/json_output/telugu_transcripts_pdfs_txt_files
/content/json_output/bengali_transcripts_pdfs_txt_files output dir
/content/processed_txt_files/bengali_transcripts_pdfs_txt_files languate dir
Processing language: bengali_transcripts_pdfs_txt_files
Created folder: /content/json_output/bengali_transcripts_pdfs_txt_files
/content/json_output/kannada_transcripts_pdfs_txt_files output dir
/content/processed_txt_files/kannada_transcripts_pdfs_txt_files languate dir
Process

## TASK 5

#### Download data and paste it project folder and give link to the app.py and run the app.py after installing all dependency.

/bin/bash: line 1: grafana-cli: command not found
